In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import ssl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import sys
sys.path.append('../')
# from fun.models import *
sns.set_style("whitegrid")
from torch.utils.data import random_split

import pandas as pd




In [2]:
import torch
from torch import nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 8, 5, padding=2)
        self.batch1 = nn.BatchNorm2d(8)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 5, padding=2)
        self.batch2 = nn.BatchNorm2d(16)
        self.fc1 = nn.Linear(16 * 8 * 8, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.batch1(x)
        x = F.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = self.batch2(x)
        x = F.relu(x)
        x = self.pool(x)

        x = torch.flatten(x, 1)

        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x


class Net_dropout(nn.Module):
    def __init__(self, dropout_rate):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 8, 5, padding=2)
        self.batch1 = nn.BatchNorm2d(8)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 5, padding=2)
        self.batch2 = nn.BatchNorm2d(16)
        self.fc1 = nn.Linear(16 * 8 * 8, 120)
        self.fc2 = nn.Linear(120, 10)

        self.drop1 = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.conv1(x)
        x = self.batch1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.drop1(x)

        x = self.conv2(x)
        x = self.batch2(x)
        x = F.relu(x)
        x = self.drop1(x)
        x = self.pool(x)

        x = torch.flatten(x, 1)

        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x


class Net_dropout_new(nn.Module):
    def __init__(self, dropout_rate):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 8, 5, padding=2)
        self.conv11 = nn.Conv2d(8, 32, 5, padding=2)
        self.batch1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(32, 64, 5, padding=2)
        self.conv22 = nn.Conv2d(64, 32, 5, padding=2)
        self.batch2 = nn.BatchNorm2d(32)

        self.conv3 = nn.Conv2d(32, 16, 5, padding=2)
        self.conv33 = nn.Conv2d(16, 8, 5, padding=2)
        self.batch3 = nn.BatchNorm2d(8)
        self.fc1 = nn.Linear(8 * 8 * 8, 120)
        self.fc2 = nn.Linear(120, 10)

        self.drop = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv11(x)
        x = self.batch1(x)
        x = F.relu(x)
        x = self.pool(x)
        # x = self.drop(x)

        x = self.conv2(x)
        x = self.conv22(x)
        x = self.batch2(x)
        x = F.relu(x)
        x = self.pool(x)
        # x = self.drop(x)

        x = self.conv3(x)
        x = self.conv33(x)
        x = self.batch3(x)
        x = F.relu(x)
        # x = self.drop(x)
        # x = self.pool(x)

        x = torch.flatten(x, 1)

        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x



In [3]:

transform = transforms.ToTensor()

dataset = torchvision.datasets.CIFAR10(
    root="../data",
    train=True,
    download=True, transform=transform)


torch.manual_seed(42)
valid_size = int(0.1 * len(dataset))
train_size = len(dataset) - valid_size


train_ds, valid_ds = random_split(dataset, [train_size, valid_size])
len(train_ds), len(valid_ds)
    

Files already downloaded and verified


(45000, 5000)

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

def train(dataloader, model, loss_fn, optimizer):

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.train()
    train_loss, correct = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    train_loss /= num_batches
    correct /= size
    print(f" Train Accuracy: {(100*correct):>0.1f}%, Train Avg loss {train_loss:>8f} \n")
    return correct, train_loss


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size

    print(f"Test Accuracy: {(100*correct):>0.1f}%, Test Avg loss: {test_loss:>8f} \n")
    return correct, test_loss
    

Using cuda device


In [6]:
#------------------------------------------------------------------
#--------------------- DO PUSZCZENIA ------------------------------
#------------------------------------------------------------------
batch_sizes = [16, 64, 256, 1024]
epochs = 40
learning_rate = 1e-3
dropout_rate = 0.2

# data = np.array([[0]*len(batch_sizes)]*epochs)
# Net_new_train_acc_history = pd.DataFrame(data, columns = batch_sizes)
# Net_new_test_acc_history = pd.DataFrame(data, columns = batch_sizes)
# Net_new_train_loss_history  = pd.DataFrame(data, columns = batch_sizes)
# Net_new_test_loss_history  = pd.DataFrame(data, columns = batch_sizes)

for batch_size in batch_sizes:

    train_dataloader = DataLoader(
            train_ds,
            batch_size=batch_size, 
            shuffle=True
            )
    test_dataloader = DataLoader(
        valid_ds, 
        batch_size=batch_size,
        shuffle=False
        )


    net = Net_dropout_new(dropout_rate=dropout_rate).to(device)
    criterion = nn.CrossEntropyLoss()

    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

    for t in range(epochs):
        
        print(f"Epoch {t+1}\n-------------------------------")
        train_acc, train_loss = train(train_dataloader, net, criterion, optimizer)
        test_acc, test_loss = test(test_dataloader, net, criterion)
#         Net_new_train_acc_history.loc[t,batch_size] = train_acc        
#         Net_new_test_acc_history.loc[t,batch_size] = test_acc
#         Net_new_train_loss_history.loc[t,batch_size] = train_loss
#         Net_new_test_loss_history.loc[t,batch_size] = test_loss

# Net_new_train_acc_history.to_csv("Net_new_train_acc_history.csv")
# !cp Net_new_train_acc_history.csv "drive/My Drive/DL/Net_net_bigger_bs_adam/"

# Net_new_test_acc_history.to_csv("Net_new_test_acc_history.csv")
# !cp Net_new_test_acc_history.csv "drive/My Drive/DL/Net_net_bigger_bs_adam/"

# Net_new_train_loss_history.to_csv("Net_new_train_loss_history.csv")
# !cp Net_new_train_loss_history.csv "drive/My Drive/DL/Net_net_bigger_bs_adam/"

# Net_new_test_loss_history.to_csv("Net_new_test_loss_history.csv")
# !cp Net_new_test_loss_history.csv "drive/My Drive/DL/Net_net_bigger_bs_adam/"

Epoch 1
-------------------------------
 Train Accuracy: 45.4%, Train Avg loss 1.489774 

Test Accuracy: 54.3%, Test Avg loss: 1.302761 

Epoch 2
-------------------------------
 Train Accuracy: 60.9%, Train Avg loss 1.094336 

Test Accuracy: 62.2%, Test Avg loss: 1.052850 

Epoch 3
-------------------------------
 Train Accuracy: 67.0%, Train Avg loss 0.927359 

Test Accuracy: 62.8%, Test Avg loss: 1.024740 

Epoch 4
-------------------------------


KeyboardInterrupt: 